In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import sklearn 
import seaborn as sns

from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

writer_r = pd.ExcelWriter(r'./SVM.xlsx')


In [ ]:
number = 0
data_path = r'./data.xlsx'
sheet_names= pd.ExcelFile(data_path).sheet_names
sheet_name = sheet_names[number]
print(sheet_name)

data = pd.read_excel(data_path,skiprows=0,sheet_name=sheet_name )
data= data.dropna()
# print(data.columns)

features=data[data.columns[2:]]
labels=data[data.columns[1]]

#Standardization
scaler=preprocessing.StandardScaler().fit(features)
features_=scaler.transform(features)
features_=pd.DataFrame(features_,columns=features.columns)
# print(features_)
print(features_.shape)

#Shuffle data order
indices = np.arange(features_.shape[0])
np.random.shuffle(indices)

features_ = features_.iloc[indices]
labels = labels.iloc[indices]

In [ ]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error,make_scorer
name='SVM'
#Define RMSE
def rmse_scorer(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))
rmse = make_scorer(rmse_scorer, greater_is_better=False)

#Define parameter value range
param_grid = {
    'C': np.arange(1, 2.1, 0.1),
    'gamma': np.arange(0, 0.030, 0.001)
}
# start prediction
SVM = SVR(kernel='rbf')
scoring = {'R2': 'r2', 'RMSE': rmse}
grid_search = GridSearchCV(estimator=SVM, param_grid=param_grid, cv=10, scoring=scoring, refit='R2', return_train_score=False)
grid_search.fit(features_, labels)

#Print best parameters and results
print("Best parameters:", grid_search.best_params_)
print("Best cross-validation R2 score: {:.4f}".format(grid_search.cv_results_['mean_test_R2'].max()))
print("Corresponding RMSE for best R2:", grid_search.cv_results_['mean_test_RMSE'][grid_search.cv_results_['mean_test_R2'].argmax()])

In [ ]:

best_params = grid_search.best_params_
best_r2_score = grid_search.cv_results_['mean_test_R2'].max()
best_rmse_score = grid_search.cv_results_['mean_test_RMSE'][grid_search.cv_results_['mean_test_R2'].argmax()]
#save parameters
data = {'name':name,
    "Best C": [best_params['C']],
    "Best Gamma": [best_params['gamma']],
    "Best R2 Score": [best_r2_score],
    "Corresponding RMSE": [best_rmse_score]
}
results_df = pd.DataFrame(data)
results_df.to_excel(writer_r,sheet_name=sheet_name)
print(results_df)

In [ ]:
#save model
import os
import pickle
path = r'./SVM'
path_model = path + '/' + sheet_name + '.pkl'
try:
    with open(path_model, 'wb') as f:
        pickle.dump(grid_search.best_estimator_, f)  
    print("Model has been successfully saved to:", path_model)
except Exception as e:
    print("Error saving model:", e)

In [ ]:
writer_r.save()
writer_r.close()